In [ ]:
# Make sure you have GPU runtime selected: Runtime -> Change runtime type -> GPU
import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


Torch: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4


In [ ]:
# Install essentials. This may take a few minutes.
# Use specific versions known to play well together. Adjust if necessary.
!pip install -q --upgrade pip
!pip install accelerate transformers datasets peft bitsandbytes safetensors sentencepiece
# For conversion later
!apt-get -qq update
!apt-get -qq install -y build-essential cmake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 47.7 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 58.9 MB/s  0:00:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 136.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 20.6 MB/s  0:00:17
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 40.1 MB/s  0:00:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 60.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 65.8 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 68.5 MB/s  0:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 27.1 MB/s  0:00:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.8 MB/s  0:0

In [ ]:
# If your model or dataset requires HF auth, run this.
from huggingface_hub import login
from google.colab import userdata
key = userdata.get('hugging_face')
login(key)

In [ ]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import bitsandbytes as bnb

# Parameters — edit as needed
BASE_MODEL = "google/medgemma-4b-it"   # change if path differs
DATASET = "ruslanmv/ai-medical-chatbot" # dataset id on HF
OUTPUT_DIR = "/content/lora_medgemma_merged"  # folder to save merged model
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
TARGET_MODULES = ["q_proj","k_proj","v_proj","o_proj","gate_proj","down_proj","up_proj"]  # try to include common proj names; adapt if model uses different names
NUM_EPOCHS = 1           # set appropriately
LEARNING_RATE = 2e-4
MICRO_BATCH_SIZE = 1     # reduce to fit memory
GRAD_ACCUM_STEPS = 8     # effective batch = micro * accum
MAX_LENGTH = 1024

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
# Load dataset
DATASET = "ruslanmv/ai-medical-chatbot" # dataset id on HF
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
import torch

# Parameters (ensure these match your setup)
MAX_LENGTH = 1024

dataset = load_dataset(DATASET)
# For demonstration assume your dataset has text field 'text' or 'prompt' or similar
# Inspect dataset column names
print("Dataset features:", dataset['train'].column_names)
# Split dataset into train and eval
dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)  # 90% train, 10% eval

# Preprocessing
# ||||||||||||||||||||||||||||||||||||||||||||||||  WRONG ||||||||||||||||||||||||||||||||||||||||||||||

# # Example: create single text column: adjust field name per dataset
# def preprocess(example):
#     # Adjust this to how dataset stores the conversation/prompt/response
#     # The dataset has 'Patient' and 'Doctor' fields that need to be combined
#     # txt = f"Description: {example['Description']}\nPatient: {example['Patient']}\nDoctor: {example['Doctor']}"
#     # formatted_text = f"Description: {example['Description']}\nPatient: {example['Patient']}\nDoctor: {example['Doctor']}"
#     messages = [
#         {"role": "system", "content": f"Description: {example['Description']}"},
#         {"role": "user", "content": example['Patient']},
#         {"role": "assistant", "content": example['Doctor']},
#     ]
#     # The tokenizer expects a string or list of strings, let's try formatting as a single string with roles
#     # formatted_text = tokenizer.apply_chat_template(messages, tokenize=False)
#     # tokens = tokenizer(formatted_text, truncation=True, max_length=MAX_LENGTH)

#     # Let's try providing a list of strings directly to the tokenizer, although apply_chat_template is preferred
#     # This might still require adjustment based on the specific tokenizer's behavior with lists
#     tokens = tokenizer([f"Description: {example['Description']}", f"Patient: {example['Patient']}", f"Doctor: {example['Doctor']}"], truncation=True, max_length=MAX_LENGTH, is_split_into_words=False)

#     # Add labels for training (for causal language modeling, labels are typically the input_ids)
#     tokens["labels"] = tokens["input_ids"]
#     return tokens

# |||||||||||||||||||||||||||||||||||||||||||||||||||||| Wrong |||||||||||||||||||||||||||||||||||||||||||||

# def preprocess(batch):
#     formatted_texts = []
#     for i in range(len(batch['Description'])):
#         messages = [
#             {"role": "system", "content": f"Description: {batch['Description'][i]}"},
#             {"role": "user", "content": batch['Patient'][i]},
#             {"role": "assistant", "content": batch['Doctor'][i]},
#         ]
#         formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
#         formatted += tokenizer.eos_token  # Add EOS if needed
#         formatted_texts.append(formatted)

#     tokens = tokenizer(
#         formatted_texts,
#         truncation=True,
#         max_length=MAX_LENGTH,
#         padding=False,  # Pad in collator
#         return_tensors=None  # List for mapping
#     )

#     # Set labels as copy of input_ids
#     tokens['labels'] = [ids[:] for ids in tokens['input_ids']]

#     # Optional: Mask prompt in labels (ignore loss on system/user parts)
#     for j in range(len(tokens['input_ids'])):
#         # Find start of assistant response (heuristic: after last user token)
#         # This requires knowing the template; adjust based on chat_template.jinja
#         # Example: Find position after "<|im_start|>assistant\n"
#         response_start = formatted_texts[j].find("<|im_start|>assistant\n") + len("<|im_start|>assistant\n")
#         if response_start > 0:
#             prompt_len = len(tokenizer(formatted_texts[j][:response_start])['input_ids'])
#             tokens['labels'][j][:prompt_len] = [-100] * prompt_len

#     return tokens

# tokenized_train = dataset['train'].map(preprocess, batched=True, remove_columns=dataset['train'].column_names)  # As above
# tokenized_eval = dataset['test'].map(preprocess, batched=True, remove_columns=dataset['test'].column_names)

# tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset['train'].column_names)

def preprocess(batch):
    formatted_texts = []
    for i in range(len(batch['Description'])):
        messages = [
            {"role": "system", "content": f"Description: {batch['Description'][i]}"},
            {"role": "user", "content": batch['Patient'][i]},
            {"role": "assistant", "content": batch['Doctor'][i]},
        ]
        # Apply chat template and add EOS token
        formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        formatted += tokenizer.eos_token
        formatted_texts.append(formatted)

    # Tokenize as a batch
    tokens = tokenizer(
        formatted_texts,
        truncation=True,
        max_length=MAX_LENGTH,  # 1024
        padding=False,  # Defer to collator
        return_tensors=None,  # Return lists for dataset mapping
    )

    # Set labels as copy of input_ids
    tokens['labels'] = [ids[:] for ids in tokens['input_ids']]

    # Optional: Mask prompt in labels to compute loss only on assistant response
    for j in range(len(tokens['input_ids'])):
        # Find start of assistant response (depends on template; adjust if needed)
        response_start = formatted_texts[j].find("<|im_start|>assistant\n")
        if response_start != -1:
            response_start += len("<|im_start|>assistant\n")
            prompt_len = len(tokenizer(formatted_texts[j][:response_start], truncation=True, max_length=MAX_LENGTH)['input_ids'])
            tokens['labels'][j][:prompt_len] = [-100] * prompt_len

    return tokens

# Apply to dataset
dataset = dataset['train'].train_test_split(test_size=0.1)  # Add validation split
tokenized_train = dataset['train'].map(preprocess, batched=True, remove_columns=dataset['train'].column_names)
tokenized_eval = dataset['test'].map(preprocess, batched=True, remove_columns=dataset['test'].column_names)

# Cache to disk to avoid re-tokenizing
tokenized_train.save_to_disk("/content/tokenized_train")
tokenized_eval.save_to_disk("/content/tokenized_eval")

README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

dialogues.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

Dataset features: ['Description', 'Patient', 'Doctor']


Map:   0%|          | 0/208101 [00:00<?, ? examples/s]

Map:   0%|          | 0/23123 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/208101 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/23123 [00:00<?, ? examples/s]

In [ ]:
tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 231224
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 25692
    })
})

In [ ]:
dataset['train'][0]

{'Description': 'Should I be concerned about having pain in the left side of my back?',
 'Patient': 'I have been having left side pain on the middle of my side but it can be in my back, chest and even my lower right side at times. No other symptoms at all. I had hernia repair with mesh 1 year ago. THe pain is tolerable just annoying. Should I be concerned?',
 'Doctor': 'Hello, The pain on one side of the back can be postural in nature. Standing for long periods and lack of adequate exercise can cause such a pain. Oral muscle relaxants and regular exercises can help. Hope I have answered your query. Let me know if I can assist you further. Take care Regards, Dr Praveen Tayal, Orthopaedic Surgeon'}

In [ ]:
# Print to verify
print("Tokenized train features:", tokenized_train.features)
print("Tokenized eval features:", tokenized_eval.features)
print("Sample tokenized train example:", tokenized_train[0])

Tokenized train features: {'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'labels': List(Value('int64'))}
Tokenized eval features: {'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'labels': List(Value('int64'))}
Sample tokenized train example: {'input_ids': [2, 2, 105, 2364, 107, 9770, 236787, 18975, 564, 577, 10099, 1003, 2963, 4331, 528, 506, 2378, 2678, 529, 1041, 1063, 236881, 108, 236777, 735, 1010, 2963, 2378, 2678, 4331, 580, 506, 6029, 529, 1041, 2678, 840, 625, 740, 577, 528, 1041, 1063, 236764, 15350, 532, 1581, 1041, 3718, 1447, 2678, 657, 2782, 236761, 2301, 1032, 10684, 657, 784, 236761, 564, 1053, 115052, 11790, 607, 18990, 236743, 236770, 1051, 4578, 236761, 558, 2209, 4331, 563, 150784, 1164, 40619, 236761, 18975, 564, 577, 10099, 236881, 106, 107, 105, 4368, 107, 9259, 236764, 669, 4331, 580, 886, 2678, 529, 506, 1063, 740, 577, 199225, 528, 4135, 236761, 52976, 573, 1440, 13443, 532, 6220, 529, 16111, 8774, 740, 440

In [ ]:
# Load model in 4-bit for memory efficiency
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True # Enable CPU offloading for 8-bit modules
)

print("Loading base model in 4-bit (this requires bitsandbytes)...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto", # Let accelerate handle device placement
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

# Prepare for k-bit training (sets gradient checkpointing, etc)
model = prepare_model_for_kbit_training(model)

# Apply LoRA (PEFT)
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)


Loading base model in 4-bit (this requires bitsandbytes)...


config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:

# Training args
training_args = TrainingArguments(
    output_dir="/content/lora_checkpoint",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    fp16=True,
    bf16=False,
    logging_steps=10,
    save_strategy="epoch",
    group_by_length=True,
    # bf16=torch.cuda.is_available() and torch.cuda.get_device_properties(0).major >= 8,  # use bfloat16 where supported

    gradient_checkpointing=True,  # For memory efficiency
    optim="paged_adamw_8bit",  # Better for quantized models
    warmup_steps=100,  # Gradual LR increase
    lr_scheduler_type="linear",  # Or "cosine"
    eval_strategy="steps",  # If adding eval_dataset
    eval_steps=250,
    max_steps=1000,  # If adding eval_dataset
    report_to="none",  # Or "wandb" for logging
)

import torch
torch.cuda.empty_cache()

In [ ]:

# data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM, not masked
    pad_to_multiple_of=8
)
training_args.eval_strategy = 'no'

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=data_collator,
)

# Start training (monitor GPU memory and logs)
trainer.train()



Step,Training Loss
10,6.845500
20,7.703400
30,7.552900
40,7.203100
50,7.360000
60,5.624800
70,5.712200
80,5.742900
90,5.486400
100,5.165500


Step,Training Loss
10,6.845500
20,7.703400
30,7.552900
40,7.203100
50,7.360000
60,5.624800
70,5.712200
80,5.742900
90,5.486400
100,5.165500


TrainOutput(global_step=1000, training_loss=5.039992839813232, metrics={'train_runtime': 2508.793, 'train_samples_per_second': 0.797, 'train_steps_per_second': 0.399, 'total_flos': 1.103369364935424e+16, 'train_loss': 5.039992839813232, 'epoch': 0.009610717872571492})

In [ ]:
# Save only LoRA adapters (optional) and merged full model later
model.save_pretrained("/content/lora_adapters")
tokenizer.save_pretrained("/content/lora_adapters")

('/content/lora_adapters/tokenizer_config.json',
 '/content/lora_adapters/special_tokens_map.json',
 '/content/lora_adapters/chat_template.jinja',
 '/content/lora_adapters/tokenizer.model',
 '/content/lora_adapters/added_tokens.json')

converting adaptor to full model to quantized

In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import PeftModel, PeftConfig
# import bitsandbytes as bnb
# from transformers import BitsAndBytesConfig

# BASE_MODEL = "google/medgemma-4b-it"   # change if path differs

# # Define the same BitsAndBytesConfig used for training
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     llm_int8_enable_fp32_cpu_offload=True # Enable CPU offloading for 8-bit modules
# )

# # Load the base model and tokenizer with the same quantization config and device_map as training
# print("Loading base model in 4-bit for merging...")
# base_model = AutoModelForCausalLM.from_pretrained(
#     BASE_MODEL,
#     quantization_config=bnb_config,
#     device_map="auto",
#     torch_dtype=torch.float16 # Use float16 for base model weights before loading 4-bit
# )
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# # Load the LoRA adapters
# print("Loading LoRA adapters...")
# lora_model = PeftModel.from_pretrained(base_model, "/content/lora_adapters")

# # Merge the LoRA adapters into the base model
# print("Merging LoRA adapters...")
# merged_model = lora_model.merge_and_unload()

# # Save the merged model
# print("Saving merged model...")
# merged_model.save_pretrained("/content/merged_model")
# tokenizer.save_pretrained("/content/merged_model")

# print("Merged model saved to /content/merged_model")

# Merge LoRA with the original base model to produce a single full model for inference
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

BASE_MODEL = "google/medgemma-4b-it"
ADAPTER_DIR = "/content/lora_adapters"
MERGED_DIR = "/content/merged_model"

# Load base (prefer fp16)
base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

In [ ]:

lora = PeftModel.from_pretrained(base, '/content/lora_adaptor')
merged = lora.merge_and_unload()

# Save merged in safetensors (recommended)
merged.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

print("Merged model saved to:", MERGED_DIR)

Merged model saved to: /content/merged_model


This quantized model should be significantly smaller and runnable on a CPU.

In [ ]:
import os
from google.colab import files

# Zip the merged model directory
!zip -r /content/merged_model.zip /content/merged_model

# Download the zip file
files.download('/content/merged_model.zip')

  adding: content/merged_model/ (stored 0%)
  adding: content/merged_model/tokenizer_config.json (deflated 97%)
  adding: content/merged_model/model-00001-of-00002.safetensors (deflated 15%)
  adding: content/merged_model/model.safetensors.index.json (deflated 96%)
  adding: content/merged_model/tokenizer.model (deflated 52%)
  adding: content/merged_model/config.json (deflated 74%)
  adding: content/merged_model/added_tokens.json (stored 0%)
  adding: content/merged_model/special_tokens_map.json (deflated 77%)
  adding: content/merged_model/chat_template.jinja (deflated 70%)
  adding: content/merged_model/model-00002-of-00002.safetensors (deflated 8%)
  adding: content/merged_model/generation_config.json (deflated 28%)
  adding: content/merged_model/tokenizer.json (deflated 83%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/lora_adapters.zip /content/lora_adapters

# Download the zip file
files.download('/content/lora_adapters.zip')


  adding: content/lora_adapters/ (stored 0%)
  adding: content/lora_adapters/tokenizer_config.json (deflated 97%)
  adding: content/lora_adapters/adapter_model.safetensors (deflated 12%)
  adding: content/lora_adapters/README.md (deflated 65%)
  adding: content/lora_adapters/tokenizer.model (deflated 52%)
  adding: content/lora_adapters/adapter_config.json (deflated 58%)
  adding: content/lora_adapters/added_tokens.json (stored 0%)
  adding: content/lora_adapters/special_tokens_map.json (deflated 77%)
  adding: content/lora_adapters/chat_template.jinja (deflated 70%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Running the merged model


In [ ]:
from ctransformers import AutoModelForCausalLM
import os

# Path to the directory where you extracted the merged model files
# Replace '/path/to/your/downloaded/merged_model' with the actual path
model_dir = '/content/merged_model'

# Assuming the main model file is named 'pytorch_model.bin'
# Check your extracted folder for the correct filename if it's different
model_file_name = 'pytorch_model.bin'

# Check if the model file exists
model_file_path = os.path.join(model_dir, model_file_name)
if not os.path.exists(model_file_path):
    print(f"Error: Model file not found at {model_file_path}")
else:
    print(f"Loading model from {model_dir}")
    try:
        # Load the merged model with ctransformers
        # Set gpu_layers=0 to ensure it runs on CPU
        # Adjust model_type based on the model architecture (e.g., "gemma")
        llm = AutoModelForCausalLM.from_pretrained(
            model_dir,
            model_file=model_file_name,
            model_type="gemma", # Replace with actual model type if not gemma
            gpu_layers=0, # Ensure it runs on CPU
            local_files_only=True # Load only from local files
        )

        print("Model loaded successfully on CPU.")

        # You can now use 'llm' for inference on CPU
        prompt = "Provide a medical diagnosis for headache and nausea."
        print(f"Prompt: {prompt}")
        print("Generating response...")
        response = llm(prompt)
        print(f"Response: {response}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
# Install bitsandbytes if not already installed
# !pip install bitsandbytes

In [ ]:
from google.colab import drive
import os
import shutil

# Remove existing files/folders in the mount point if any
if os.path.exists('/content/drive'):
    shutil.rmtree('/content/drive')
os.makedirs('/content/drive')

drive.mount('/content/drive')

Mounted at /content/drive


Moving files from colab to drive

In [ ]:
import os

# Define the source and destination paths
source_dir = '/content/merged_model'
# Replace 'My Drive/merged_model' with the desired path in your Google Drive
destination_dir = '/content/drive/MyDrive/merged_model'

# Create the destination directory in Drive if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Use shell command to copy the files
!cp -r "$source_dir" "$destination_dir"

print(f"Copied contents of {source_dir} to {destination_dir}")

Copied contents of /content/merged_model to /content/drive/MyDrive/merged_model
